# How To Train Your ResNet

<img src="https://raw.githubusercontent.com/zhanghang1989/ResNet-Matconvnet/master/figure/resnet_cifar.png" width="640" border="1"/>

At this point we have assembled the classes into a small framework that resides under the `/lib/ootf` folder. We are going to deploy a state-of-the-art CNN architecture, that combines increased accuracy with small memory footprint.

`(Objective: Train a ResNet20 on CIFAR10 with ~90% accuracy, understand residual networks through the source code. Time: 40 mins)`

In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')
print()

# Check GPU
import tensorflow as tf
print(tf.test.gpu_device_name())
     
import sys
sys.path.append('/content/gdrive/My Drive/Colab Notebooks/OOT2019/lib')

# Display the system path elements
for sFolder in sys.path:
  print(sFolder)

We import the classes from the `ootf` package. Starting with the settings for the training process.

In [0]:
import numpy as np
import tensorflow as tf
from datetime import datetime
from ootf.tfcifar10 import TFDataSetCifar10
from ootf.resnet import ResNet
from ootf.base import ModelState


#  ........... Settings ...........
RANDOM_SEED         = 2019
DATA_FOLDER         = "/content/gdrive/My Drive/Colab Notebooks/OOT2019/data/tfcifar10"
TRAINING_INTERVAL   = 250
VALIDATION_INTERVAL = 1000
SAVE_INTERVAL       = 2000

# (He,2015) We start with a learningrate of 0.1, divide it by 10 at 32k and 48k iterations, and terminate training at 64k iterations
INITIAL_LEARNING_RATE = 0.1
INITIAL_MOMENTUM = 0.9
LR_SCHEDULE_STEPS_ORIGINAL = [32000,48000,64000]


# Makes the random initialization reproducible
tf.reset_default_graph() 
tf.set_random_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
print("Random seed set to %d" % RANDOM_SEED)

The data feed thread will load TFRecords from the files that were created in `/data/tfcifar10`

In [0]:
# ........... Loads data / Creates the data feed ...........
oDataFeed = TFDataSetCifar10(DATA_FOLDER)
LR_SCHEDULE_STEPS = LR_SCHEDULE_STEPS_ORIGINAL
print(" |_ Learning rate schedule:%s. Batch sizes, training: %d testing: %d." % (LR_SCHEDULE_STEPS, oDataFeed.TrainBatchSize, oDataFeed.TestBatchSize))

## Non-standard settings 
We increase the training batch size to x4 compared with the standard size. Please keep in mind that such change require hyperparameter tunning, to achieve the optimum training result.

In [0]:
from ootf.nn import NeuralNetwork

# We are using x4 the default batch size
oDataFeed.TrainBatchSize *= 4
oDataFeed.TestBatchSize *= 5

# It will need half an hour on GPU to achieve 89.7% accuracy 
LR_SCHEDULE_STEPS = [6000, 7000, 8000] 

print(" |_ Learning rate schedule:%s. Batch sizes, training: %d testing: %d." % (LR_SCHEDULE_STEPS, oDataFeed.TrainBatchSize, oDataFeed.TestBatchSize))

## Residual CNN architecture

We create the ResNet model. The second parameter in the constructor defines the type of ResNet. For ResNet20 we have 3 stacks of 3 modules that is 3\*3\*2+2=20. For ResNet32 we have 3 stacks of 5 modules 3\*5\*2+2=32. Please refer to the original paper [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) for other architectures. 

<img src="https://raw.githubusercontent.com/torch/torch.github.io/master/blog/_posts/images/resnets_modelvariants.png" width="600" border="1"/>

### Implementation Notes
*   The `:ResidualModule` implements the original(reference) paper
*   When downsampling is performed with stride 2 the skip connection is implemented in method `ResidualConnection()` as an average pooling layer. It can be replaced with a learnable convolutional layer.


In [0]:
import numpy as np
from tensorflow.python import control_flow_ops

from ootf.cnn import ConvolutionalNeuralNetwork
from ootf.base import Evaluator
from ootf.nn import NeuralNetwork


# =======================================================================================================================
class ResidualModule(object):
    # --------------------------------------------------------------------------------------------------------
    def __init__(self, p_oParent, p_nFeatures, p_nStrideOnInput):
        #......................... |  Instance Attributes | .............................
        # // Aggregates \\ 
        self.Parent         = p_oParent
        
        # // Settings \\
        self.Features       = p_nFeatures
        self.StrideOnInput  = p_nStrideOnInput
        
        # // Tensors \\
        self.Input          = None
        self.InputFeatures  = None
        self.Output         = None
        #................................................................................
    # --------------------------------------------------------------------------------------------------------
    def ResidualConnection(self, p_tModuleInput):
        with tf.variable_scope("RESIDUAL"):
          # Spatial downsampling in the first convolutional layer of the module and the skip connection
          if (self.StrideOnInput > 1):
              tSpatialDownsampling = self.Parent.AveragePool(p_tModuleInput, (2,2), (2,2), p_bIsPadding=False)
          else:
              tSpatialDownsampling = p_tModuleInput
                  
          # Pad features if dimensionality of features increases 
          if (self.InputFeatures != self.Features):
              tResidual = self.Parent.PadFeatures(tSpatialDownsampling, self.Features, p_sName="pad_skip")
          else:
              tResidual = tf.identity(tSpatialDownsampling, "skip")
  
        return tResidual
    # --------------------------------------------------------------------------------------------------------
    def Function(self, p_tX):
        self.Input          = p_tX
        self.InputFeatures  = self.Input.get_shape().as_list()[3]

        sModuleName = "RES%d" % (len(self.Parent.Modules) + 1)
        with tf.variable_scope(sModuleName):
            tResidual = self.ResidualConnection(p_tX)
            
            tA = self.Parent.Convolutional(p_tX, self.Features, (3,3), (self.StrideOnInput, self.StrideOnInput), p_nPadding=1, p_bHasBias=False, p_tInitializer=tf.initializers.he_uniform()) #HE INITIALIZATION
            tA = self.Parent.BatchNormalization(tA)
            tA = tf.nn.relu(tA)
            
            tA = self.Parent.Convolutional(tA, self.Features, (3,3), (1,1), p_nPadding=1, p_bHasBias=False, p_tInitializer=tf.initializers.he_uniform())
            tA = self.Parent.BatchNormalization(tA)
            
            tY = tf.nn.relu(tA + tResidual)
                
            self.Output = tY    
        return tY
    # --------------------------------------------------------------------------------------------------------      
# =======================================================================================================================





   
#==================================================================================================  
class ResNet(ConvolutionalNeuralNetwork):  
    #------------------------------------------------------------------------------------
    def __init__(self, p_nFeatures=[(32,32,3),16,32,64,10], p_nStackSetup=[5,5,5], p_oDataFeed=None):
        #......................... |  Instance Attributes | .............................
        # // Composite object collections \\
        self.Modules = []
        
        # // Aggregates  \\
        self.DataFeed = p_oDataFeed
                
        # // Architectural Hyperparameters \\
        self.StackSetup = p_nStackSetup
        self.DownSampling = [False, True, True, True]

        # // Learning Hyperparameters \\
        #self.Momentum     = 0.9
        self.WeightDecay  = 1e-4
        self.DropOutRate  = 0.5

        # // Tensors \\
        self.Input            = None
        self.Targets          = None
        self.TargetsOneHot    = None
        
        self.Logits           = None
        self.Prediction       = None
        self.PredictedClass   = None
        self.Correct          = None
        self.Accuracy         = None
        
        self.WeightDecayCost  = None
        self.CCECost          = None
        self.CostFunction     = None
        #................................................................................
        
        # Invoke the inherited logic from ancestor :NeuralNetwork
        super(ResNet, self).__init__(p_nFeatures)
    # --------------------------------------------------------------------------------------------------------
    def __getModuleStrides(self, p_nStackIndex):
        nModuleCount = self.StackSetup[p_nStackIndex]
    
        if  self.DownSampling[p_nStackIndex]: 
            nStrides = [2]
        else:
            nStrides = [1]
        nStrides = nStrides + [1]*(nModuleCount -1)
        
        return nStrides        
    # --------------------------------------------------------------------------------------------------------
    def CreateInput(self):
        assert self.DataFeed is not None, "This model works with a TFRecords data feed"
      
        tTrainImageBatch, tTrainLabelBatch = self.DataFeed.TrainingBatches()
        tTestImageBatch, tTestLabelBatch = self.DataFeed.TestingBatches()
        
        tImages, tLabels = control_flow_ops.cond(self.IsTraining,
            lambda: (tTrainImageBatch, tTrainLabelBatch),
            lambda: (tTestImageBatch, tTestLabelBatch))
    
        self.Input = tImages
        self.Targets   = tLabels   
        
        return self.Input, self.Targets 
    def Feed(self, p_nBatchFeatures, p_nBatchTargets=None, p_nLearningRate=None, p_bIsTraining=False):
      oDict = dict()
      
      oDict[self.Input]      = p_nBatchFeatures
      oDict[self.IsTraining] = p_bIsTraining
      
      if p_bIsTraining:
        oDict[self.Targets]      = p_nBatchTargets
        oDict[self.LearningRate] = p_nLearningRate
        oDict[self.DropOutKeepProb] = self.DropOutRate
      else:
        # When the model is trained the neurons are not dropped out
        oDict[self.DropOutKeepProb] = 1.0
        
      return oDict
    # --------------------------------------------------------------------------------------------------------
    def Predict(self, p_oSession, p_oSubSet):   
      nPredictedClasses = np.zeros(p_oSubSet.Labels.shape, np.uint32)
      for nRange, nSamples, nLabels, _ in p_oSubSet:
        nPrediction = p_oSession.run(self.Prediction, feed_dict=self.Feed(nSamples, nLabels))
        nPredictedClasses[nRange] = np.argmax(nPrediction, axis=1).astype(np.uint32)
      
      return nPredictedClasses
    # --------------------------------------------------------------------------------------------------------
    def Evaluate(self, p_oDataSubSet=None, p_nBatchSize=1000, p_oProcess=None, p_oSession=None):
        assert self.DataFeed is not None, "This model works with a TFRecords data feed"
      
        n_val_samples = 10000
        val_batch_size = self.DataFeed.TestBatchSize
        
        n_val_batch = int(  np.ceil(n_val_samples / val_batch_size) )
        
        #val_logits = np.zeros((n_val_samples, 10), dtype=np.float32)
        val_labels = np.zeros((n_val_samples), dtype=np.int64)
        
        pred_labels = np.zeros((n_val_samples), dtype=np.int64)
        val_losses = []
        for i in range(n_val_batch):
            fetches = [self.Prediction, self.Targets, self.CostFunction]
            fetches.append(self.PredictedClass)
            
            if p_oDataSubSet is None:
                oFeedDict = {self.IsTraining: False}
            else:
                oFeedDict = dict()
                oFeedDict[self.Input]       = p_oDataSubSet.Patterns[i * val_batch_size:(i + 1) * val_batch_size,...]
                oFeedDict[self.Targets]     = p_oDataSubSet.Labels[i * val_batch_size:(i + 1) * val_batch_size,...]
                oFeedDict[self.IsTraining]  = False
                
            if p_oSession is not None:
              session_outputs = p_oSession.run(fetches, oFeedDict)
            else:   
              session_outputs = self.Session.run(fetches, oFeedDict)
            
            pred_labels[i * val_batch_size:(i + 1) * val_batch_size] = session_outputs[3]
            val_labels[i * val_batch_size:(i + 1) * val_batch_size] = session_outputs[1]
            val_losses.append(session_outputs[2])
        
        oEval = Evaluator(val_labels, pred_labels)
        # print(nAvgBatchesAccuracy)
        if p_oProcess is None:
            print("Accuracy:", oEval.Accuracy)
        else:
            p_oProcess.Print("Accuracy:", oEval.Accuracy)
        print(oEval.ConfusionMatrix)
                            
        val_loss     = float(np.mean(np.asarray(val_losses)))    
        val_accuracy = oEval.Accuracy
        
        return val_loss, val_accuracy
    # --------------------------------------------------------------------------------------------------------
    def CreateModel(self):
      nClassCount = self.Features[-1]
        
      with tf.variable_scope("NeuralNet"):
        #tInput, tTargets = 
        self.CreateInput()
        
        with tf.variable_scope("Targets"):
          self.TargetsOneHot = tf.one_hot(self.Targets, depth=nClassCount, dtype=tf.float32)

        with tf.variable_scope("Stem"):
            tA = self.Convolutional(self.Input, self.Features[1], (3,3), (1,1), p_bIsPadding=True, p_bHasBias=False, p_tInitializer=tf.initializers.he_uniform())
            tA = self.BatchNormalization(tA)
            tA = tf.nn.relu(tA)

        # Creates the stacks of residual modules
        nStackCount = len(self.StackSetup)
        for nStackIndex in range(0, nStackCount):
            nModuleStrides = self.__getModuleStrides(nStackIndex)
            
            sStackName = "Stack%d" % (nStackIndex + 1)
            with tf.variable_scope(sStackName):
                print("  [%s]" % sStackName)              
                for nModuleIndex, nModuleInputStride in enumerate(nModuleStrides):
                    oResBlock = ResidualModule(self, self.Features[nStackIndex + 1], nModuleInputStride)
                    tA = oResBlock.Function(tA)
                    
                    self.Modules.append(oResBlock)
                    print(" |_ Res%d" % (nModuleIndex+1), oResBlock.Input, oResBlock.Output)                                      
                  
        # For each output feature,  averages the values in its spatial activation table. 
        # This results feature activation vector for each image
        tA = self.GlobalAveragePooling(tA)
        
        # Softmax layer (classifier)
        self.Logits     = self.FullyConnected(tA, nClassCount)
        self.Prediction = tf.nn.softmax(self.Logits)       
        
        # Predictions and accuracy
        with tf.variable_scope("Predictions"):
            self.PredictedClass     = tf.argmax(self.Prediction , 1, output_type=tf.int32)
            self.Correct            = tf.equal(self.PredictedClass, tf.cast(self.Targets, tf.int32))
            self.Accuracy           = tf.reduce_mean(tf.cast(self.Correct , tf.float32), name='accuracy')
        
      # Prepare cost function tensors for training
      self.DefineCostFunction()
    # --------------------------------------------------------------------------------------------------------
    def DefineCostFunction(self):
        with tf.variable_scope("Cost"):
            # Multiclass categorical cross entropy (CCE) loss
            tLoss = tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.Logits, labels=self.TargetsOneHot)
            self.CCECost  = tf.reduce_mean(tLoss, name="cce")
            
            # L2 weight decay regularization
            tL2LossesConv = [tf.nn.l2_loss(tKernel) for tKernel in self.ConvWeights]
            tL2LossesFC   = [tf.nn.l2_loss(tWeight) for tWeight in self.FCWeights]
            tL2LossesAll  = tL2LossesConv + tL2LossesFC
            tWeightDecay = tf.constant(self.WeightDecay, tf.float32, name="weight_decay")
            self.WeightDecayCost = tf.multiply(tWeightDecay, tf.add_n(tL2LossesAll), name="l2")

            # Total cost
            self.CostFunction = tf.identity(self.CCECost + self.WeightDecayCost, "total_cost")
    # --------------------------------------------------------------------------------------------------------
#==================================================================================================  



# ...................... Creates the model ......................
sModelName = "ResNet20"

print("\n[>] Creating %s model ..." % sModelName)

#(He,2015) Reported accuracy for ResNet20 in Table 6:91.25%               
oModel = ResNet([(32,32,3),16,32,64,10], [3,3,3], p_oDataFeed=oDataFeed)                    

## Create a custom training operation
Tensorflow comes with easy to use optimizers that can create gradient tensors, through automatic derivation. The easiest is to call the `.minimize()` method on an optimizer. The example bellow illustrates how to create a custom training operation, by having the gradient tensors available, that is useful for more advanced optimization methods. 

In [0]:
# ...................... Trains the model ......................        
# Creates the training operation
print("\n[>] Creating training operation ...")        
oUpdateOperations = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(oUpdateOperations):
    with tf.variable_scope("CustomOptimizer"):
        # Gets the gradients for the cost function w.r.t. the trainable model parameters
        tModelParameters = tf.trainable_variables()
        tGradients       = tf.gradients(oModel.CostFunction, tModelParameters, name='gradients')
        print(" |__ A total of %d trainable model parameters have %d gradients." % (len(tModelParameters), len(tGradients)))
        print("     |__ Convolutional Kernels:%d" % len(oModel.ConvWeights))
        print("     |__ Convolutional Biases:%d" % len(oModel.ConvBiases))
        print("     |__ Fully Connected Weights:%d" % len(oModel.FCWeights))        
        print("     |__ Fully Connected Biases:%d" % len(oModel.FCBiases))
        
        # Prioritize dependencies
        oUpdateOperations   = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(oUpdateOperations):
            tOptimizer      = tf.train.MomentumOptimizer(oModel.LearningRate, INITIAL_MOMENTUM)
            tBackPropagate  = tOptimizer.apply_gradients(zip(tGradients, tModelParameters), global_step=oModel.GlobalStep)
            with tf.control_dependencies([tBackPropagate]):
                tTrainOp = tf.no_op(name='train_op')


We initialize the session and create some helper objects.

In [0]:
# Initializes the session and its variables
print("\n[>] Initializing...")
oSession = tf.Session()
oSession.run(tf.global_variables_initializer())

# Creates helper objects
oState = ModelState(oSession, "/content/gdrive/My Drive/Colab Notebooks/OOT2019/models/%s" % sModelName)              
  
print(" |__ GPU:", tf.test.gpu_device_name())    

## Reproducible random weight initialization

We can run this unit test to ensure that setting the random seed in Tensorflow results in reproducible initial weights.  Nevertheless the training process in the GPU is not deterministic due to the low level implementation of the 2D convolution.


No exceptions should be raised by the following test.

In [0]:
# --------------------------------------------------------------------------------------------------------  
def testReproducibility(p_oModel, p_oSession):
  tW_CONV19 = p_oModel.ConvWeights[-1]
  tW_FC1 = p_oModel.FCWeights[-1]
  
  nW_CONV19 = tW_CONV19.eval(p_oSession)
  nW_FC1 = tW_FC1.eval(p_oSession)
  
  print("[%s] Initial Weights: Mean=%.6f Std=%.6f" % (tW_CONV19.name, np.round(np.mean(nW_CONV19), 6), np.round(np.std(nW_CONV19), 6)) )
  print("[%s] Initial Weights: Mean=%.6f Std=%.6f" % (tW_FC1.name, np.round(np.mean(nW_FC1), 6), np.round(np.std(nW_FC1), 6)) )
  
  assert tW_CONV19.name == "NeuralNet/Stack3/RES9/CONV19/w:0", "Architecture is not ResNet20"
  assert tW_FC1.name == "NeuralNet/FC1/w:0", "Architecture is not ResNet20"  
  
  assert (np.round(np.mean(nW_CONV19), 6) - 0.000051) <= 1e-5, "CONV19 kernel initial values have different mean than expected"
  assert (np.round(np.std(nW_CONV19), 6) - 0.058901) <= 1e-5, "CONV19 kernel initial values have different std than expecteds"

  assert (np.round(np.mean(nW_FC1), 6) - 0.000182) <= 1e-5, "FC1 weight initial values have different mean than expected"
  assert (np.round(np.std(nW_FC1), 6) - 0.163457) <= 1e-5, "FC1 weight initial values have different std than expecteds"
# --------------------------------------------------------------------------------------------------------    
  

testReproducibility(oModel, oSession)

## Main training loop
The main training loop begins by starting the threads that feed data to an input queue. Running the training operation tensor requests the next minibatch of data automatically. Only the training flag and the learning rate are passed with a feed dictionary. At specific intervals i) more tensor values are retrieved, ii) the model is evaluated using the whole testing subset and iii) the model is saved.

In [0]:
print("[>] Started training at %s ..." % datetime.now() )
      
# Starts the data feed threads
tCoordinator = tf.train.Coordinator()       
tThreads = tf.train.start_queue_runners(sess=oSession, coord=tCoordinator)

# Training loop   
nMaxSteps  = LR_SCHEDULE_STEPS[-1]
nLR = INITIAL_LEARNING_RATE 
nEpochIndex = 0
nLastValidationAccuracy = 0.0
for nStepNumber in range(1, nMaxSteps+1):
    bIsLastStep              = (nStepNumber == nMaxSteps) 
    bIsTrainingPrintInterval = ((nStepNumber % TRAINING_INTERVAL) == 0) or bIsLastStep
    bIsValidationInterval    = ((nStepNumber % VALIDATION_INTERVAL) == 0) or bIsLastStep
    bIsSaveInterval          = ((nStepNumber % SAVE_INTERVAL) == 0) or bIsLastStep
    
    oTensorList = [tTrainOp, oModel.CostFunction]
    if bIsTrainingPrintInterval:
        oTensorList += [oModel.WeightDecayCost, oModel.Accuracy]
    nRunResult = oSession.run(oTensorList, feed_dict={oModel.IsTraining: True, oModel.LearningRate: nLR})

    # Print training status
    if bIsTrainingPrintInterval:
        nTrainingTotalLoss, nTrainingL2Loss, nTrainingAccuracy, = nRunResult[1:]
        print('%s [%d] Iteration %6d | LR=%.6f | ERR=%.6f (WD=%.6F) | ACC=%.4f' % (datetime.now(), nEpochIndex+1, nStepNumber, nLR
                                                , nTrainingTotalLoss, nTrainingL2Loss, nTrainingAccuracy))

    # Validation on the whole testing set
    if bIsValidationInterval:
        print('[%d] Evaluating...' % (nEpochIndex+1))
        oModel.Evaluate(p_oSession=oSession)
        nEpochIndex += 1
        
    # Save the model state
    if bIsSaveInterval:
        print("[>] Saving after %s epochs" % SAVE_INTERVAL)
        oState.Save()
    
    # Implements the learning rate schedule
    if nStepNumber in LR_SCHEDULE_STEPS[:-1]:
      nLR = nLR * 0.1
      print("[>] Changed learning rate to %.5f" % nLR)
      oState.Save()
                
# Terminates the data feed threads gracefully
tCoordinator.request_stop()
tCoordinator.join(tThreads)  

oSession.close()
print("[>] Finished training at %s ..." % datetime.now() )

## ResNet inference

In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from ootf.base import DataSubSet
from ootf.cifar10 import DataSetCifar10

import requests
from PIL import Image
from io import BytesIO
import joblib

MODEL_STATE_NAME = "/content/gdrive/My Drive/Colab Notebooks/OOT2019/models/ResNet20" 
oNet = oModel

# ... False predictions ...
IMAGE1 = "https://upload.wikimedia.org/wikipedia/commons/thumb/8/8b/Husky_on_San_Francisco_sidewalk.jpg/220px-Husky_on_San_Francisco_sidewalk.jpg"
IMAGE2 = "https://upload.wikimedia.org/wikipedia/commons/thumb/b/b1/Hellenic_Hound_aka_Hellinikos_Ichnilatis.jpg/220px-Hellenic_Hound_aka_Hellinikos_Ichnilatis.jpg"

# ... Correct predictions ...
IMAGE3 = "https://vignette.wikia.nocookie.net/dog-breed4080/images/0/0a/GREATER-SWISS-MOUNTAIN-DOG.jpg/revision/latest/scale-to-width-down/230?cb=20171221134620"
IMAGE4 = "https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Cat03.jpg/1200px-Cat03.jpg"

# Downloads the images from their URLs
sImageURLs = [IMAGE1, IMAGE2, IMAGE3, IMAGE4]
sImageLabels = ["husky", "greek harehound", "swiss mountain dog", "cat"]
nImageBatch  = np.zeros((len(sImageURLs), 32, 32, 3), np.uint8)
for nIndex,sImageURL in enumerate(sImageURLs):
  oResponse = requests.get(sImageURL)
  nImage = Image.open(BytesIO(oResponse.content))
  nImage = nImage.resize((32,32), Image.ANTIALIAS)
  nImageIn = np.array(nImage)
  nImageBatch[nIndex,:,:,:] = nImageIn[:,:,:]

# Z-score standardization for bots sets
oDict = joblib.load(r"/content/gdrive/My Drive/Colab Notebooks/OOT2019/data/tfcifar10/meanstd.pkl")
nPixelMean = oDict["mean"]
nPixelStd = oDict["std"]
nImageBatchStandardized = (nImageBatch.astype(np.float32) - nPixelMean) / nPixelStd

# Initializes the session and its variables
print("\n[>] Initializing...")
oSession = tf.Session()
oSession.run(tf.global_variables_initializer())

# Creates helper objects
oState = ModelState(oSession, MODEL_STATE_NAME) 
oClassNamesDict = DataSetCifar10(DataSubSet, "/content/gdrive/My Drive/Colab Notebooks/OOT2019/data/cifar10").ClassNames

# Loads the saved state and runs inference
if oState.Load():
  oImageFeed = oNet.Feed(nImageBatchStandardized, p_bIsTraining=False)
  nPredictedProbs, nPredictedClass = oSession.run([oNet.Prediction, oNet.PredictedClass], feed_dict=oImageFeed)
  
  for nSampleIndex in range(0,nPredictedClass.shape[0]):
    sPredictedClass = oClassNamesDict[nPredictedClass[nSampleIndex]]
    print("\n[>] The image of a %s is predicted as %s (%d)" % (sImageLabels[nSampleIndex], sPredictedClass, nPredictedClass[nSampleIndex]))
    print(" |___ Probabilities:%s" % ["%s:%.2f%%" % (oClassNamesDict[nIndex], nPredictedProbs[nSampleIndex, nIndex]*100.0) for nIndex in range(10)])
    oImagePlot = plt.imshow(nImageBatch[nSampleIndex,...])
    plt.show()
  
oSession.close()